### SQLChatMessageHistory
* 채팅메시지를 sql데이터베이스 저장하는 클래스

** 주요특징 **
* sql 데이터베이스를 사용해 채팅기록을 영구적으로 저장
* 세션 id를 통한 대화구분 및 관리가능
* sqlalchemy를 사용해 다양한 데이터 베이스사용가능
* 메시지 추가 및 조회 crud가능

* session_id : 사용자 이름, 이메일 같이 고유 식별자
* connection : 데이터베이스 연결을지정하는 문자


In [20]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="",
    connection="sqlite:///sqlite.db"
)

In [21]:
chat_message_history.add_user_message(
    "안녕?만나서 반가워 나는 bear야"
)

In [22]:
chat_message_history.add_ai_message(
    "안녕 bear, 만나서 반가워"
)

In [23]:
chat_message_history.messages

[HumanMessage(content='안녕?만나서 반가워 나는 bear야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 bear, 만나서 반가워', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕?만나서 반가워 나는 bear야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 bear, 만나서 반가워', additional_kwargs={}, response_metadata={})]

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),

        MessagesPlaceholder(variable_name="chat_history"),

        ("human", "{question}")
    ]
)

chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o") | StrOutputParser()

In [2]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chains import LLMChain

# ChatPromptTemplate 인스턴스 생성
prompt = ChatPromptTemplate(
    messages=[
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

# 사용할 질문과 대화 기록을 포함한 변수
variables = {
    "question": "안녕하세요, 어떤 도움이 필요하신가요?",
    "chat_history": []  # 빈 대화 기록으로 초기화
}

# 메시지를 포맷팅
formatted_prompt = prompt.format_messages(**variables)

# LLMChain 생성
llm_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=prompt  # 원래의 프롬프트를 사용
)

# 체인 실행
response = llm_chain.run(question=variables["question"], chat_history=variables["chat_history"])

# 결과 출력
print(response)

ModuleNotFoundError: No module named 'langchain_core.chains'

In [15]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [16]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_field = [
    ConfigurableFieldSpec(
        id="user_id", # 설정값의 고유 식별자
        annotation=str, # 설정값의 데이터 타입
        name="USER ID", # 설정값의 이름
        description="Unique identifier for a user", # 설정값에 대한 설명
        default="", # 기본값
        is_shared=True # 여러 대화에서 공유되는 값인지 여부
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="CONVERSATION ID",
        description="Unique identifier for a conversation",
        default="",
        is_shared=True
    )
]

In [17]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history, # 대화기록을 가져오는 함수
    input_messages_key="question", # 입력 메시지 키
    history_messages_key="chat_history", # 대화 기록 메시지의 키
    history_factory_config = config_field # 대화기록 조회시 참고할 파라미터
)

In [18]:
config  = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [19]:
chain_with_history.invoke({"question" : "안녕 반가워 내이름은 bear야"}, config)

NameError: name 'SQLChatMessageHistory' is not defined

In [13]:
config  = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

chain_with_history.invoke({"question" : "내이름이 뭐라고?"}, config)

NameError: name 'SQLChatMessageHistory' is not defined

In [14]:

config  = {"configurable": {"user_id": "user2", "conversation_id": "conversation1"}}

chain_with_history.invoke({"question" : "내이름이 뭐라고?"}, config)

NameError: name 'SQLChatMessageHistory' is not defined